In [17]:
import sys
sys.path.append("..")

In [18]:
from src.download_data import download_kaggle_dataset

dataset_name = "mlg-ulb/creditcardfraud"
temp_dir = download_kaggle_dataset(dataset_name)

import os
print(f"Files in {temp_dir}: {os.listdir(temp_dir)}")

Dataset URL: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
Dataset downloaded to temporary directory: temp_download
Files in temp_download: ['X_train_scaled.csv', 'creditcard.csv', 'scaler.pkl', 'X_test_scaled.csv']


In [19]:
import pandas as pd

file_path = os.path.join(temp_dir, "creditcard.csv")
data = pd.read_csv(file_path)

In [20]:
X = data.drop("Class", axis=1)
y = data["Class"]

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [22]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [23]:
import joblib

joblib.dump(scaler, os.path.join(temp_dir, "scaler.pkl"))  # Save scaler for inference
pd.DataFrame(X_train_scaled).to_csv(os.path.join(temp_dir,"X_train_scaled.csv"), index=False)
pd.DataFrame(X_test_scaled).to_csv(os.path.join(temp_dir,"X_test_scaled.csv"), index=False)

In [24]:
print(f"Files in {temp_dir}: {os.listdir(temp_dir)}")

Files in temp_download: ['X_train_scaled.csv', 'creditcard.csv', 'scaler.pkl', 'X_test_scaled.csv']
